# Generate Permutation Importance (i.e., rank) from SHAP files

In [2]:
import shap
import numpy as np
import pickle
import glob
import pathlib
import subprocess
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import Affine
from osgeo import gdal

In [2]:
import os
def is_non_zero_file(fpath):  
    return os.path.isfile(fpath) and os.path.getsize(fpath) > 0

In [1]:
from pathlib import Path
path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_2_bands_bandListFile_1_trials_20240225/754873093086037409/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::754873093086037409.keras[2, 10].shap_values0to50'
Path(path).touch()

In [10]:
print(path, ' size == ', os.path.getsize(path))
if ( os.path.getsize(path) == 0):
    print(path, ' size still == ', os.path.getsize(path))

/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_2_bands_bandListFile_1_trials_20240225/754873093086037409/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::754873093086037409.keras[2, 10].shap_values0to50  size ==  0
/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_2_bands_bandListFile_1_trials_20240225/754873093086037409/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::754873093086037409.keras[2, 10].shap_values0to50  size still ==  0


In [1]:
#bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/random_sets_20240203_collection-ALL_max_occurrences-50_batch_size-10_num_samples-3227_bandLen-447.pkl.test'
bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/random_sets_2.bak'
setFile = open(bandListFile, "rb")
random_sets_r = pickle.load(setFile)
setFile.flush()
setFile.close()
print('initial # of subsets in input file:', len(random_sets_r))
print(random_sets_r)

NameError: name 'pickle' is not defined

In [6]:
shapPrefix = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials'
#'\]\.model:' 
processedModelFiles = glob.glob(shapPrefix + '**/**/MODELS/*].model',
                       recursive=True)
print('# of model files already processed:', len(processedModelFiles), '\n', processedModelFiles)


# of model files already processed: 2 
 ['/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials/446420826688885424/MODELS/MLP_SGD_7_layers_1024_units_MAE::446420826688885424.keras[[196, 235, 227, 410, 39, 263, 214, 256, 442, 60]].model', '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials/446420826688885424/MODELS/MLP_SGD_7_layers_1024_units_MAE::446420826688885424.keras[[284, 31, 307, 262, 71, 204, 92, 390, 248, 164]].model']


In [8]:
shapPrefix = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials'
processedShapFiles = glob.glob(shapPrefix + '**/**/PERMUTATION_IMPORTANCE_VALUES/*.shap_values0to50',
                       recursive=True)
print('# of shap files already processed:', len(processedShapFiles), '\n', processedShapFiles)


# of shap files already processed: 2 
 ['/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials/446420826688885424/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::446420826688885424.keras[[196, 235, 227, 410, 39, 263, 214, 256, 442, 60]].shap_values0to50', '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/exp_10_bands_10_trials/446420826688885424/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::446420826688885424.keras[[284, 31, 307, 262, 71, 204, 92, 390, 248, 164]].shap_values0to50']


In [2]:
!gdalinfo /explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices.tif | grep Index > /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_hyperspectral_indices-bandnames.csv
!more /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_hyperspectral_indices-bandnames.csv

    Index Name=ARI
    Index Name=CAI
    Index Name=CRI550
    Index Name=CRI700
    Index Name=EVI
    Index Name=EVI2
    Index Name=fPAR
    Index Name=LAI
    Index Name=MCTI
    Index Name=MSI
    Index Name=NDII
    Index Name=NDLI
    Index Name=NDNI
    Index Name=NDVI
    Index Name=NDWI
    Index Name=NIRv
    Index Name=PRIn
    Index Name=PRIw
    Index Name=SAVI
    Index Name=WBI
    Index Name=Albedo


In [3]:
!gdalinfo /explore/nobackup/projects/ilab/projects/AGB/MLBS-2015/2015_NEON_D07_MLBS_DP3_547000_4141000_reflectance.tif | grep Wavelength > /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/2015_NEON_D07_MLBS_DP3_547000_4141000_reflectance.csv
#!more /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled-named.csv

In [6]:
hyperspectral_indices = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_hyperspectral_indices-bandnames.csv'
neon_reflectance = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/2015_NEON_D07_MLBS_DP3_547000_4141000_reflectance.csv'

In [9]:
import csv

hyperspectral_indices_bandnames = []
with open(hyperspectral_indices) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='=')
    line_count = 0
    for row in csv_reader:
#        print(row[1])
        hyperspectral_indices_bandnames.append(row[1])
print(hyperspectral_indices_bandnames)

['ARI', 'CAI', 'CRI550', 'CRI700', 'EVI', 'EVI2', 'fPAR', 'LAI', 'MCTI', 'MSI', 'NDII', 'NDLI', 'NDNI', 'NDVI', 'NDWI', 'NIRv', 'PRIn', 'PRIw', 'SAVI', 'WBI', 'Albedo']


In [10]:
with open(neon_reflectance) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='=')
    line_count = 0
    for row in csv_reader:
#        print(row[1])
        hyperspectral_indices_bandnames.append(row[1])
print(hyperspectral_indices_bandnames)

['ARI', 'CAI', 'CRI550', 'CRI700', 'EVI', 'EVI2', 'fPAR', 'LAI', 'MCTI', 'MSI', 'NDII', 'NDLI', 'NDNI', 'NDVI', 'NDWI', 'NIRv', 'PRIn', 'PRIw', 'SAVI', 'WBI', 'Albedo', '383.072', '388.0806', '393.0891', '398.0977', '403.1062', '408.1147', '413.1233', '418.1318', '423.1403', '428.1489', '433.1574', '438.1659', '443.1745', '448.183', '453.1916', '458.2001', '463.2086', '468.2172', '473.2257', '478.2342', '483.2428', '488.2513', '493.2598', '498.2684', '503.2769', '508.2855', '513.294', '518.3025', '523.3111', '528.3196', '533.3281', '538.3367', '543.3452', '548.3537', '553.3623', '558.3708', '563.3794', '568.3879', '573.3964', '578.405', '583.4135', '588.422', '593.4306', '598.4391', '603.4476', '608.4562', '613.4647', '618.4733', '623.4818', '628.4903', '633.4989', '638.5074', '643.5159', '648.5245', '653.533', '658.5415', '663.5501', '668.5586', '673.5672', '678.5757', '683.5842', '688.5928', '693.6013', '698.6098', '703.6184', '708.6269', '713.6354', '718.644', '723.6525', '728.6611'

In [ ]:
# read csv file to a list of dictionaries
with open(hyperspectral_indices_bandnames, 'r') as file:
    csv_reader = csv.DictReader(file)
    data = [row for row in csv_reader]
print(data[1])

In [ ]:
from datetime import datetime
start_time = datetime.now()
import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)
print("Your Computer Name is:" + hostname)
print("Your Computer IP Address is:" + IPAddr)

time_elapsed = datetime.now() - start_time
print('Time elapsed on node [{}] = (hh:mm:ss.ms) {}'.format(hostname, time_elapsed))

In [ ]:
file = "/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_Reflectance_reflectance_warp.tif"
raster = rasterio.open(file)

plt.imshow(raster.read(1), cmap="BrBG")
plt.title("Temperature")
plt.show()
raster.close()

In [ ]:
# ds = gdal.Open(file)  

# for band in range(1, ds.RasterCount):
#     print(ds.GetRasterBand(band).GetMetadata())

In [ ]:
with rasterio.open(file) as src:
    print(src.descriptions)
    print(src.meta)
    print(src.profile)

In [ ]:
file = "/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_Reflectance_reflectance_warp.tif"
raster = rasterio.open(file)

In [ ]:
raster.profile

In [ ]:
raster.profile['height']

In [ ]:
raster_band = raster.read(1)
raster_band

In [ ]:
scaled_raster_band = np.divide(raster_band, 10000)
scaled_raster_band

In [ ]:
# Loop through raster bands
file = "/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_Reflectance_reflectance_warp.tif"
raster_out = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif'
raster = rasterio.open(file)

scaled_bands = []
for band_no in range(1, raster.count + 1):
    # Read band
    raster_band = raster.read(band_no)

    scaled_raster_band = np.divide(raster_band, 10000)

    # Append padded raster band to list
    scaled_bands.append(scaled_raster_band)

with rio.open(raster_out, 'w', **out_meta) as dest:
    for band_nr, src in enumerate(scaled_bands, start=1):
        dest.write(src, band_nr)

In [ ]:
!ls '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_7bands/20240115/600584965588330414/MODELS/MLP_7bands_2_epochs::600584965588330414.keras[[2, 16]].model/variables'

In [ ]:
import sys
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion')
sys.path.append('/explore/nobackup/people/gtamkin/dev/AGB/monte-carlo/monte-carlo')
# from mc.model.config.MpfConfig import MpfConfig

In [ ]:
!ls -alRt /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/20240112/789617681324607598/MODELS/ | grep '\]\.model:' | wc -l
!ls -alRt /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/20240112/789617681324607598/MODELS/ | grep '\]\.test' | wc -l
!ls -alRt /explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/20240112/789617681324607598/PERMUTATION_IMPORTANCE_VALUES/ | grep shap |wc -l

In [ ]:
bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/random_sets/random_sets_20240111_shrink_False_num_samples_1088_bandLen_447'
random_sets_r = pickle.load(open(bandListFile, "rb"))
print(len(random_sets_r))

bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/aggregate/random_sets_20240111_shrink_False_num_samples_1088_bandLen_447.bak'
random_sets_orig = pickle.load(open(bandListFile, "rb"))
print(len(random_sets_orig))


In [ ]:

file1 = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[382, 116, 40, 358, 112, 346, 137, 76, 136, 83]].shap_values0to50'
file0 = '/_explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[382, 116, 40, 358, 112, 346, 137, 76, 136, 83]].shap_values0to50'
file2 = "/explore/nobackup/people/gtamkin/dev/AGB/mpf-model-factories/MultiPathFusion/multi_path_fusion/notebooks/glenn"
result = subprocess.run(["/usr/bin/ls", file1], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print('result: ', result)
print('result.returncode: ', result.returncode)
print('result.stdout: ', result.stdout)
print('result.stderr: ', result.stderr)


In [ ]:
bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/20240108/random_sets_20240108_shrink_False_num_samples_1196_bandLen_426'
random_sets_r = pickle.load(open(bandListFile, "rb"))

indexListIntStr = ''
print(str(random_sets_r[0]))
count = 0
for band in random_sets_r[0]: 
    if (count == 0):
        indexListIntStr = indexListIntStr + band
    else:
            indexListIntStr = indexListIntStr + ', ' + band   
    count = count + 1
print('indexListIntStr:', indexListIntStr) 

shapFile =  "'/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[" + str(indexListIntStr) + "]].shap_values0to50'"

print(pathlib.Path(str(shapFile)).is_file())
print (os.path.isfile(str(shapFile)))

shapFile2 =  "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[" + str(indexListIntStr) + "]].shap_values0to50"
result = subprocess.run(["/usr/bin/ls", shapFile2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print('\nresult: ', result)
print('result.returncode: ', result.returncode)
print('result.stdout: ', result.stdout)
print('result.stderr: ', result.stderr)

print('\n', shapFile)
!/usr/bin/ls -alt $shapFile



In [ ]:
# # pip install gdal-utils
# import osgeo_utils.gdal_merge

# img_list = ['/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_hyperspectral_indices.tif', 
#             '/explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_Reflectance_reflectance_warp.tif']
# output_file_path = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate-separate.tif'
# parameters = ['', '-o', output_file_path] + img_list + ['-separate', '-ot', 'Float32']
# osgeo_utils.gdal_merge.main(parameters)

In [ ]:
bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/20240108/random_sets_20240108_shrink_False_num_samples_1196_bandLen_426'
random_sets_r = pickle.load(open(bandListFile, "rb"))

indexListIntStr = ''
print(str(random_sets_r[0]))
count = 0
for band in random_sets_r[0]: 
    if (count == 0):
        indexListIntStr = indexListIntStr + band
    else:
        if (int(band) < 10):
            indexListIntStr = indexListIntStr + ',   ' + band
        elif (int(band) < 100):
            indexListIntStr = indexListIntStr + ', ' + band   
        else:
            indexListIntStr = indexListIntStr + ', ' + band   
    count = count + 1
print('indexListIntStr:', indexListIntStr) 

shapFile =  "'/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[" + str(indexListIntStr) + "]].shap_values0to50'"
print(shapFile)
!/usr/bin/ls -alt $shapFile

shapFile2 =  "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[" + str(indexListIntStr) + "]].shap_values0to50"
result = subprocess.run(["/usr/bin/ls", shapFile2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print('result: ', result)
print('result.returncode: ', result.returncode)
print('result.stdout: ', result.stdout)
print('result.stderr: ', result.stderr)


In [ ]:
bandListFile = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/20240108/random_sets_20240108_shrink_False_num_samples_1196_bandLen_426'
random_sets_r = pickle.load(open(bandListFile, "rb"))
# print(shapFile)
#indexListStr = '\\[\\'
indexListStr = ''
print(str(random_sets_r[0]))
count = 0
for band in random_sets_r[0]: 
    if (count == 0):
        indexListStr = indexListStr + str(band)
    else:
        if (int(band) < 10):
            indexListStr = indexListStr + '\\,\\   ' + str(band) 
        elif (int(band) < 100):
            indexListStr = indexListStr + '\\,\\ ' + str(band)    
        else:
            indexListStr = indexListStr + '\\,\\ ' + str(band)  
    count = count + 1
print('indexListStr:', indexListStr) 

goodList = '382\,\ 116\,\ 40\,\ 358\,\ 112\,\ 346\,\ 137\,\ 76\,\ 136\,\ 83'
print('goodList    :', goodList)
if (indexListStr != goodList):
    print('No: indexListStr != goodList')
else:
    print('Yes: indexListStr == goodList')
        
    print('indexList2:', goodList)
    shapFile =  '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[' + goodList + ']].shap_values0to50'
#    shapFile =  "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::642759939845152617.keras[[' + goodList + ']].shap_values0to50"
    print(shapFile)
    !/usr/bin/ls -alt $shapFile

    result = subprocess.run(["/usr/bin/ls", shapFile], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print('result: ', result)
    print('result.stdout: ', result.stdout)
    print('result.stderr: ', result.stderr)
    
    print(pathlib.Path(shapFile).is_file())
    if (os.path.isfile(str(shapFile))):
        print(' found: ', shapFile)
    else:
        print(' did not find: ', shapFile)
#indexList1 = '\[\[181\,\ 302\,\ 342\,\ 333\,\ 389\,\ 300\]\]'
indexList1 = '181\,\ 302\,\ 342\,\ 333\,\ 389\,\ 300'
print('\nindexList1:', indexList1)
shapFile =  '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::423994812500766108.keras[[' + indexList1 + ']].shap_values0to50'
#shapFile =  '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/MLP_SGD_7_layers_1024_units_MAE::423994812500766108.keras' + indexList1 + '.shap_values0to50'
print(shapFile)
!ls -alt $shapFile


## Read batch of configuration files for experiment, which contain pointers to other run-time artifacts (e.g., SHAP files)

In [ ]:
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_all21bands/01022024/random/CONFIG"
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_random21bands/0102212024/CONFIG"
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/01032024/CONFIG"
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/20240105b/423994812500766108/PERMUTATION_IMPORTANCE_VALUES"
cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/20240105b/423994812500766108/CONFIG"
# Returns a list of names in list files. 
print("Using glob.glob()") 
cfg_files = glob.glob(cfg_dir + '**/*.cfg',  
                   recursive = True) 
for file in cfg_files: 
    print(file) 
print('num cfg files: ', len(cfg_files))  


In [ ]:
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_all21bands/01022024/random/CONFIG"
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_random21bands/0102212024/CONFIG"
#cfg_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/01032024/CONFIG"
shap_dir = "/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/20240105b/423994812500766108/PERMUTATION_IMPORTANCE_VALUES"
# Returns a list of names in list files. 
print("Using glob.glob()") 
shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                   recursive = True) 
# for file in cfg_files: 
#     print(file) 
print('shap_files[0]: ', shap_files[0], '\nnum shap files: ', len(shap_files))  


In [ ]:
Bands = !/panfs/ccds02/app/modules/anaconda/platform/x86_64/rhel/8.6/3-2022.05/envs/ilab-tensorflow/bin/gdalinfo /explore/nobackup/projects/ilab/data/AGB/test/beta_pmm/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0/MLBS_2018_Reflectance_reflectance_warp.tif | grep Band | wc -l
print('Bands: ', Bands)

In [ ]:
# set to max number of hyperspectral indices in experiment
#bandLen = 21
bandLen = int(Bands[0])
print(bandLen)
bandOccurenceArr = np.zeros(bandLen).astype(int)
print('bandOccurenceArr: \n', bandOccurenceArr)
bandAbsSumArr = np.zeros(bandLen)
bandMaxArr = np.zeros(bandLen)
bandMinArr = np.zeros(bandLen)
bandAvgArr = np.zeros(bandLen)

indexArray = np.zeros(bandLen).astype(int)
for i in range(bandLen):
    indexArray[i] = indexArray[i] + i
print('indexArray: \n', indexArray, len(indexArray))


## Load SHAP values for each batch in experiment.  
### Aggregate results and rank features 

In [3]:
import re
s = text = shap_vals_path = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/20240105b/423994812500766108/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::423994812500766108.keras[[181, 302, 342, 333, 389, 300]].shap_values0to50'
shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))

num_bands = len(shap_values0to50[0][0])
print('num_bands :', num_bands)

print('len(shap_values0to50[0][0][0] (', len(shap_values0to50),',', len(shap_values0to50[0]),',', len(shap_values0to50[0][0]),')')

# Transpose test data for shap_values generation
import pandas as pd
df = pd.DataFrame(data=shap_values0to50[0])
#print(df[:12])

shap_values0to50_abs = np.abs(shap_values0to50[0])
df = pd.DataFrame(data=shap_values0to50_abs)
#print(df[:12])

for k in range(0, num_bands):
    print('column[',k,']',df[k].sum())

#print(df[0:3], df[0:3].sum())
#print(df[:], df[:].sum())

df = pd.DataFrame(data=shap_values0to50[0][0])
#print(df[:12])

df = pd.DataFrame(data=shap_values0to50[0][0][0].flatten())
#print(df[:12])


FileNotFoundError: [Errno 2] No such file or directory: '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_full_hyperspectral/20240105b/423994812500766108/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE::423994812500766108.keras[[181, 302, 342, 333, 389, 300]].shap_values0to50'

In [ ]:
start = left = '[['
end = right = ']]'
print(text[text.index(left)+len(left):text.index(right)])

bandStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
#print(bandStr)
#bandList = [bandStr]
#print(bandList)
#bandList = [int(e) if e.isdigit() else e for e in bandStr.split(',')]
bandList = [e for e in bandStr.split(',')]
print(bandList, len(bandList))

In [ ]:
def reset_stats():
    # set to max number of hyperspectral indices in experiment
    #bandLen = 21
    bandLen = int(Bands[0])
#    print(bandLen)
    bandOccurenceArr = np.zeros(bandLen).astype(int)
#    print('\ninitial bandOccurenceArr: \n', bandOccurenceArr)
    bandAbsSumArr = np.zeros(bandLen)
#    print('initial bandAbsSumArr: \n', bandOccurenceArr)
    bandMaxArr = np.zeros(bandLen)
    bandMinArr = np.zeros(bandLen)
    bandAvgArr = np.zeros(bandLen)

    indexArray = np.zeros(bandLen).astype(int)
    for i in range(bandLen):
        indexArray[i] = indexArray[i] + i
    #print('indexArray: \n', indexArray, len(indexArray))


In [ ]:
    #  Walk across bins [0,4]
    print('len(shap_values0to50[0][0][0] (', len(shap_values0to50),',', len(shap_values0to50[0]),',', len(shap_values0to50[0][0]),')')
#    shap_values_raw = shap_values0to50[_bin]
#    shap_values_abs = np.abs(shap_values0to50[_bin])
#    print('\nshap_values_abs.shape  : ', shap_values_abs.shape)
 #   print('shap_values_raw.shape  : ', shap_values_raw.shape)
    shap_values0to50_abs = np.abs(shap_values0to50[0])
    df_abs = pd.DataFrame(data=shap_values0to50_abs)
    for k in range(0, num_bands):
        print('column[',k,']', df_abs[k].sum())

    for _bin in range(0, 5):
        print('\n _bin:', _bin)
        # print('len(shap_values0to50[',_bin,'][0])', len(shap_values0to50[_bin]),' X ', len(shap_values0to50[_bin][0]))
#        shap_values_raw = shap_values0to50[_bin]
#        shap_values_abs = np.abs(shap_values0to50[_bin])
#        print('\shap_values0to50[_bin].shape  : ', shap_values0to50[_bin].shape)
#        print('shap_values_raw.shape  : ', shap_values_raw.shape)
        
        for _row in range(0, 50):
            print('\n _bin:', _bin, ' _row:', _row)
            shap_values_raw_row = shap_values0to50[_bin][_row]
            shap_values_abs_row = np.abs(shap_values0to50[_bin][_row])
            print('shap_values_raw bin [', _bin, '] x row [', _row, '] :', shap_values_raw_row)
            print('shap_values_abs bin [', _bin, '] x row [', _row, '] :', shap_values_abs_row)

            shap_values_abs_row_sum_axis0 = np.sum(shap_values_abs_row[0], axis=0)
            print('shap_values_abs_row_sum_axis0  : ', shap_values_abs_row_sum_axis0)

            for _cell in range(0, 6):
                o = 9
            
            
#             shap_values_abs_row = np.abs(shap_values0to50[_bin][_row])
#             print('shap_values_abs_row  : ', shap_values0to50[_bin][_row])

#             shap_values_abs_row_sum_axis0 = np.sum(shap_values_abs_row, axis=0)
#             print('shap_values_abs_row_sum_axis0  : ', shap_values_abs_row_sum_axis0)

#         print(shap_values_abs[_row])
#             shap_values_abs_sum_axis0 = np.sum(np.abs(shap_values0to50[_bin]), axis=0)
#             print('shap_values_abs_sum_axis0.shape  : ', shap_values_abs_sum_axis0.shape)

#             shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
#             print('L->H :', shap_values_abs_sum_argsort)

                
#            for _cell in range(0, 6):
                # if first column of row is negative, print debug
#                print('\n _bin:', _bin, ' _row:', _row, '_cell:', _cell)
                # if ((shap_values_raw[_row][_cell] < 0) and (_row == 0) and (_cell == 0)):
                #     debug = True
                #     print('\nshap_values_raw bin [', _bin, '] x row [', _row, '] :', shap_values_raw[_row])
                #     print('shap_values_raw bin [', _bin, '] x row [', _row, '] x cell[', _cell, ']:', shap_values_raw[_row][_cell])
                #     print('shap_values_abs bin [', _bin, '] x row [', _row, '] :', shap_values_abs[_row])
                #     print('shap_values_abs bin [', _bin, '] x row [', _row, '] x cell[', _cell, ']:', shap_values_abs[_row][_cell])
                #     debug = False
                    
#         shap_values_abs_sum_axis0 = np.sum(shap_values_abs, axis=0)
#         print('shap_values_abs_sum_axis0.shape  : ', shap_values_abs_sum_axis0.shape)

#         shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
#         print('L->H :', shap_values_abs_sum_argsort[0])



In [ ]:
first_time = True
index = 0
# Reload config
start = left = '[['
end = right = ']]'
reset_stats()

for shap_vals_path in shap_files:
    
    index = index+1
    num_shap_files = len(shap_files)
    bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
    bandList = [e for e in bandListStr.split(',')]
    
    # Reload shap values
    shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
    num_bins = len(shap_values0to50)
    num_rows = len(shap_values0to50[0])
    num_bands = num_cols = len(shap_values0to50[0][0])
    print('\nProcessing shap file: index ', index,' out of ', num_shap_files, ':', os.path.basename(shap_vals_path))
    print('num_bins (', num_bins, ') num_rows (', num_rows, ') num_bands = num_cols(', num_bands, ') bandList(', bandList)
    
    #  Walk across bins [0,4]
    for _bin in range(0, num_bins):
#        print('len(shap_values0to50[',_bin,'])', len(shap_values0to50[_bin]))
        shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])
        print('shap_values0to50_bin_abs.shape  : ', shap_values0to50_bin_abs.shape)
        
        # get sum of absolute value each column
        df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)

        shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)
        print('shap_values_abs_sum_axis0.shape  : ', shap_values_abs_sum_axis0.shape)

        shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
        print('L->H:\n', shap_values_abs_sum_argsort)
        
        # print sum of absolute value each column
        for _col in range(0, num_cols):
            print('band[',bandList[_col].strip(),'] column[',_col,']', df_shap_values0to50_bin_abs[_col].sum())

        #
        break;
        #  Walk across bins [0,4]
        for i in range(0, num_bands):

             #  Walk across bins [0,4]
            for j in range(0, num_bands):
                   # get band from band list
                bandNum = int(bandList[i])

        #         print('i', str(i), '\nbandNum: ', bandNum, shap_values_abs_sum_axis0[0]
        # #              ' shap_values_abs_sum_axis0[', str(bandNum-1),'][', str(i),']', 
        # #              str(shap_values_abs_sum_axis0[bandNum-1][i]), 
        #                ' bands: ', str(bandList), ' num_bands: ', str(num_bands), ' bandNum: ', str(bandNum))

                # increment Occurence value in the cell that contains the band number
                bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1

                # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
                # this allows us to keep a running total shap values per feature for downselection later....
        #        bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])
                bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])

                occurences_axis0 = np.sum(shap_values_abs, axis=0)

                # calculate average value of each band by dividing the absolute value by the number of occurences
                bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])

                with np.printoptions(precision=5, suppress=True):
                    print('\nbandNum: ', bandNum)
        #            print('bandOccurenceArr: ', bandOccurenceArr)
                    print('bandAbsSumArr: ',bandAbsSumArr)
        #             print('bandAvgArr: ',bandAvgArr)

    if (index > 2):
        break;
        
#shap.summary_plot(shap_values0to50, X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

In [ ]:
        # get band from band list

bandNum = int(bandList[i-1])
print(bandNum)

# print('i', str(i), ' shap_values_abs_sum_axis0[', str(bandNum-1),'][', str(i),']', str(shap_values_abs_sum_axis0[bandNum-1][i]), 
#        ' bands: ', str(cfg_obj.bandList), ' num_bands: ', str(num_bands), ' bandNum: ', str(bandNum))

# increment Occurence value in the cell that contains the band number
bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1

# add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
# this allows us to keep a running total shap values per feature for downselection later....
#        bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])
bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[i][i])

occurences_axis0 = np.sum(shap_values_abs, axis=0)

# calculate average value of each band by dividing the absolute value by the number of occurences
bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])


In [ ]:
#cfg = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/exp_all21bands/01012024/bandListFile-save/CONFIG/Exp_all21bands::725617110916401361.keras[[20, 11, 13, 15, 21, 7, 9, 17, 1, 4]].cfg'
# cfg = cfg_files[0]
# cfg_obj = pickle.load(open(cfg, "rb"))
# print(cfg_obj)
# shap_vals_path = cfg_obj.shap_path
# print('shap_vals_path:', shap_vals_path)

In [ ]:
# shap_vals_path = cfg_obj.shap_path
# shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
# print(len(shap_values0to50[0][0]), cfg_obj.bandList)

In [ ]:
first_time = True
# Reload config
for file in cfg_files:
    print('Processing file: \n', file)
    cfg_obj = pickle.load(open(file, "rb"))   
    cfg_path = cfg_obj.configFile
#    num_bands = len(cfg_obj.bandList)
    shap_vals_path = cfg_obj.shap_path
    modelPath = cfg_obj.model_path
    # archive_test_generator_id = cfg_obj.trial_path
    keyword = cfg_obj.bandList

#     # Reload test data
#     test_generator = pickle.load(open(archive_test_generator_id, "rb"))

#     # Transpose test data for shap_values generation
#     import pandas as pd
#     df = pd.DataFrame(data=test_generator.file_x_stack)
#     X = df.transpose()
#     X.iloc[:, :]

    # Reload shap values
    shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
    num_bands = len(shap_values0to50[0][0])
                    
    shap_values_abs = np.abs(shap_values0to50)

    shap_values_abs_sum_axis0 = np.sum(shap_values_abs, axis=0)

    shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
    
    shap_values_abs_sum_argsort_r = np.argsort(-shap_values_abs_sum_axis0)
    
    print("\n**** feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))")
    print('shap_vals_path: ', str(shap_vals_path))
    print('bands: ', str(cfg_obj.bandList))
    print('sum  : ', shap_values_abs_sum_axis0[0])
    print('L->H :', shap_values_abs_sum_argsort[0])
    print('H->L :', shap_values_abs_sum_argsort_r[0])
    
    for i in range(num_bands):

        # get band from band list
        bandNum = cfg_obj.bandList[i]
        
        # print('i', str(i), ' shap_values_abs_sum_axis0[', str(bandNum-1),'][', str(i),']', str(shap_values_abs_sum_axis0[bandNum-1][i]), 
        #       ' bands: ', str(cfg_obj.bandList), ' num_bands: ', str(num_bands), ' bandNum: ', str(bandNum))
        
        # increment Occurence value in the cell that contains the band number
        bandOccurenceArr[bandNum-1] = bandOccurenceArr[bandNum-1] + 1
    
        # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
        # this allows us to keep a running total shap values per feature for downselection later....
        bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + shap_values_abs_sum_axis0[bandNum-1][i])
    
        occurences_axis0 = np.sum(shap_values_abs, axis=0)
    
        # calculate average value of each band by dividing the absolute value by the number of occurences
        bandAvgArr[bandNum-1] = (bandAbsSumArr[bandNum-1] / bandOccurenceArr[bandNum-1])

    
    with np.printoptions(precision=5, suppress=True):
        print(indexArray)
        print(bandOccurenceArr)
        print(bandAbsSumArr)
        print(bandAvgArr)
        
#shap.summary_plot(shap_values0to50, X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

# ***Derive aggregated feature order***

**================================================= RANK ==================================**

In [ ]:

print('\nDerive feature order based on average:')
avg_shap_values_abs_sum_argsort_r = np.argsort(bandAvgArr)
print('avg cumulative: \nmin()' , bandAvgArr.min(), '\nmax()', bandAvgArr.max(), '\n', bandAvgArr) 
print('avg cumulative feature order L->H :' , avg_shap_values_abs_sum_argsort_r) 

print('\nDerive feature order based on aggregate sum:')
cum_shap_values_abs_sum_argsort_r = np.argsort(bandAbsSumArr)
print('raw cumulative: \nmin()' , bandAbsSumArr.min(), '\nmax()', bandAbsSumArr.max(), '\n', bandAbsSumArr) 
print('raw cumulative feature order L->H :' , cum_shap_values_abs_sum_argsort_r) 



**================================================= RANK ==================================**

# *FYI...*

## Calculate additional SHAP values (sum, abs, sort)

In [ ]:
# from _beeswarm.py.   
#        feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))


def permutation_importance(shap_values):
#    print("\nshap_values:\n", shap_values.shape)
#    print("\nshap_values:\n", shap_values, shap_values.dtype, shap_values.shape)

    shap_values_sum_axis1 = np.sum(shap_values, axis=1)
#    print("\nshap_values_sum_axis1: np.sum(shap_values_sum, axis=1) [sum across row]\n", shap_values_sum_axis1)

    shap_values_sum_axis0 = np.sum(shap_values, axis=0)
#    print("\nshap_values_sum_axis0 : np.sum(shap_values, axis=0) [sum down column]\n", shap_values_sum_axis0)

    shap_values_sum_argsort = np.argsort(shap_values_sum_axis0)
#    print ("\nshap_values_sum_argsort: np.argsort(shap_values_sum_axis0)  [sum down column]\n", shap_values_sum_argsort)

    shap_values_abs = np.abs(shap_values)
#    print("\nshap_values_abs: (np.abs(nd_a))\n", shap_values_abs)

    shap_values_abs_sum_axis1 = np.sum(shap_values_abs, axis=1)
#    print("\nshap_values_abs_sum_axis1: np.sum(shap_values_abs, axis=1) [sum across row]\n", shap_values_abs_sum_axis1)

    shap_values_abs_sum_axis0 = np.sum(shap_values_abs, axis=0)
#    print("\nshap_values_abs_sum_axis0: np.sum(shap_values_abs, axis=0) [sum down column]\n", shap_values_abs_sum_axis0)

    shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)
#    print("\n**** feature_order = np.argsort(np.sum(np.abs(shap_values), axis=0))")
#    print ("shap_values_abs_sum_argsort: np.argsort(shap_values_abs_sum_axis0)  [sum down column]\n *", shap_values_abs_sum_argsort)

    feature_order = shap_values_abs_sum_argsort
#    if not (shap_values_abs_sum_argsort[0] == feature_order[0]): raise AssertionError
#    print('\nfeature_order:\n', feature_order)
    
    return shap_values_sum_axis1, shap_values_sum_axis0, shap_values_sum_argsort, \
        shap_values_abs, shap_values_abs_sum_axis1, shap_values_abs_sum_axis0, shap_values_abs_sum_argsort, feature_order

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
a = [[1, 2, 3, 4, 5, 6, -7], [1, 2, 3, 4, 5, -29, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7], [-1, 2, 3, 4, 5, 6, 7]]
print(np.array(a))
permutation_importance(np.array(a))
    # return shap_values_sum_axis1, shap_values_sum_axis0, shap_values_sum_argsort, \
    #     shap_values_abs, shap_values_abs_sum_axis1, shap_values_abs_sum_axis0, shap_values_abs_sum_argsort, feature_order

In [ ]:
print(i, bandNum, bandAbsSumArr[bandNum-1], shap_values_abs_sum_axis0[bandNum-1][i])
print('sum  : ', shap_values_abs_sum_axis0[0][1])


## Interrogate configuration object (optional)

In [ ]:
# Reload config
cfg_obj = pickle.load(open(cfg_files[0], "rb"))   
print(cfg_obj)
print(cfg_obj.bandList)
#mpfConfig = (MpfConfig) cfg_obj
#print(mpfConfig)

In [ ]:
print(cfg_obj.model_config)
print(cfg_obj.model_config['model_name'])

In [ ]:
print(cfg_obj.mlflow_config)
print(cfg_obj.mlflow_config['EXPERIMENT_NAME'])

In [ ]:
print(cfg_obj.X)
print(cfg_obj.data_generator_config)
print(cfg_obj.data_generator_config['data_path'])
print(cfg_obj.model_path)

In [ ]:
cfg_path = cfg_obj.configFile
num_bands = len(cfg_obj.bandList)
shap_vals_path = cfg_obj.shap_path
modelPath = cfg_obj.model_path
archive_test_generator_id = cfg_obj.trial_path
keyword = cfg_obj.bandList

print(cfg_path)
print(modelPath)
print(archive_test_generator_id)
print(shap_vals_path)
print(cfg_obj.bandList, num_bands)

## Load test data to run additional plots (optional)

In [ ]:
# Reload test data
test_generator = pickle.load(open(archive_test_generator_id, "rb"))

# Transpose test data for shap_values generation
import pandas as pd
df = pd.DataFrame(data=test_generator.file_x_stack)
X = df.transpose()
X.iloc[:, :]
print(X.shape, X.shape[0], X.shape[1])

In [ ]:
# Reload shap values
shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))

In [ ]:
# print according to most in particular bin:  0 < -1.0, 1 < -0.5, 2 < 0, 3 < 0.5, 4 < 1.0
print('bin value 0: [-1.0, -0.5]] = summary_plot(shap_values0to50[0]') 
shap.summary_plot(shap_values0to50[4], X.iloc[0:50, :], plot_type="bar", feature_names=keyword)

In [ ]:
shap.summary_plot(shap_values0to50, X, plot_type="bar", feature_names=keyword)

In [ ]:
shap_values_np = np.array(shap_values0to50)
shap_values_np[0].shape

In [ ]:
df = pd.DataFrame(data=shap_values0to50[4])
df.head(5)

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
#print(np.array(shap_values50))
_shap_values50_sum_axis1, _shap_values50_sum_axis0, _shap_value50s_sum_argsort, \
        _shap_values50_abs, _shap_values50_abs_sum_axis1, _shap_values50_abs_sum_axis0, _shap_values50_abs_sum_argsort, _feature50_order = \
        permutation_importance(np.array(shap_values0to50[0]))

print ('shap_values0to50[0] feature_order:\n', _feature50_order.shape, '\n', _feature50_order)

_shap_values50_sum_axis1, _shap_values50_sum_axis0, _shap_value50s_sum_argsort, \
        _shap_values50_abs, _shap_values50_abs_sum_axis1, _shap_values50_abs_sum_axis0, _shap_values50_abs_sum_argsort, _feature50_order = \
        permutation_importance(np.array(shap_values0to50))

print ('\nshap_values0to50[0] feature_order:\n', _feature50_order[0].shape, '\n', _feature50_order[0], '\n',  _shap_values50_abs_sum_axis0[0])
print ('\nshap_values0to50 feature_order:\n', _feature50_order.shape, '\n', _feature50_order, '\n',  _shap_values50_abs_sum_axis0)

In [ ]:
# example row/column statistics with test matrix that is easy to predict.  Note that argsort goes from low to high.
#print(np.array(shap_values0to50))
permutation_importance(np.array(shap_values0to50))

In [ ]:
#!/usr/bin/env python3
from functools import partial
from itertools import repeat
from multiprocessing import Pool, freeze_support

def func(a, b):
    print(a+b)
    return a + b

def main():
    a_args = [1,2,3]
    second_arg = 1
    with Pool() as pool:
        L = pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
        M = pool.starmap(func, zip(a_args, repeat(second_arg)))
        N = pool.map(partial(func, b=second_arg), a_args)
        assert L == M == N

if __name__=="__main__":
    freeze_support()
    main()

In [ ]:
import multiprocessing

pool = multiprocessing.Pool(processes=6)
pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
pool.close()
pool.join()